In [ ]:
import numba
import numpy as np
import holoviews as hv
hv.extension("bokeh","matplotlib",logo=False)
@numba.njit()
def interp2dNumba(inA,outA,j):
    for i2 in range(outA.shape[0]):
        f2=i2/j
        l2=int(f2)
        f2=f2-l2
        for i1 in range(outA.shape[1]):
            f1=i1/j
            l1=int(f1)
            f1=f1-l1
            outA[i2,i1]=(1.-f2)*(1.-f1)*inA[l2,l1]+\
              (1.-f2)*f1*inA[l2,l1+1]+\
              (f2)*(1.-f1)*inA[l2+1,l1]+f2*f1*inA[l2+1,l1+1]
    
def interp2d(inA,j):
    ns=((inA.shape[0]-1)*j+1,(inA.shape[1]-1)*j+1)
    outA=np.zeros(ns,np.float32)
    interp2dNumba(inA,outA,j)
    return outA


ina=np.random.rand(10,10)
outa=interp2d(ina,10)

combo=(hv.Image(ina,bounds=(0,0,1,1)).opts(cmap="gist_gray")+hv.Image(outa,bounds=(0,0,1,1)).opts(cmap="gist_gray")).opts(toolbar=None)
hv.renderer("bokeh").save(combo,"linear",fmt="png")


In [ ]:
from bokeh.io import export_png
import syntheticModel
p=np.ndarray((100,100))


p1 = np.linspace(0,1,100, endpoint=False)
p2 = np.linspace(0,1,100, endpoint=False)
p = np.stack(np.meshgrid(p1, p2, indexing='ij'), -1)
p = np.reshape(p, (-1, 2)).T

x=np.reshape(syntheticModel.model_noise.fractal(p,n_octaves=1),(100,100))
y=np.reshape(syntheticModel.model_noise.fractal(p,frequency=2,n_octaves=1),(100,100))
z=np.reshape(syntheticModel.model_noise.fractal(p,frequency=3,n_octaves=1),(100,100))


def set_toolbar_autohide(plot,element):
    bokeh_plot=plot.state
    bokeh_plot.toolbar.autohide=True
    bokeh_plot.toolbar.logo=None
    
o={"cmap":"gist_gray"}


freq=(hv.Image(x,bounds=(0,0,1,1)).opts(**o)+\
hv.Image(y,bounds=(0,0,1,1)).opts(**o)+\
hv.Image(z,bounds=(0,0,1,1)).opts(**o)).opts(toolbar=None)
hv.renderer("bokeh").save(freq,"freq",fmt="png")

freq

In [ ]:
import syntheticModel.model_noise
x=np.reshape(syntheticModel.model_noise.fractal(p,n_octaves=1),(100,100))
y=np.reshape(syntheticModel.model_noise.fractal(p,n_octaves=2),(100,100))
z=np.reshape(syntheticModel.model_noise.fractal(p,n_octaves=3),(100,100))
octaves=(hv.Image(x,bounds=(0,0,1,1)).opts(cmap="gist_gray")+\
hv.Image(y,bounds=(0,0,1,1)).opts(cmap="gist_gray")+\
hv.Image(z,bounds=(0,0,1,1)).opts(cmap="gist_gray")).opts(toolbar=None)

hv.renderer("bokeh").save(octaves,"octaves",fmt="png")
octaves

In [ ]:
x=np.reshape(syntheticModel.model_noise.fractal(p,persistence=.2),(100,100))
y=np.reshape(syntheticModel.model_noise.fractal(p,persistence=.5),(100,100))
z=np.reshape(syntheticModel.model_noise.fractal(p,persistence=.8),(100,100))
pers=(hv.Image(x,bounds=(0,0,1,1)).opts(cmap="gist_gray")+\
hv.Image(y,bounds=(0,0,1,1)).opts(cmap="gist_gray")+\
hv.Image(z,bounds=(0,0,1,1)).opts(cmap="gist_gray")).opts(toolbar=None)
hv.renderer("bokeh").save(pers,"pers",fmt="png")
pers

In [ ]:
import syntheticModel
hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,400,50],os=[0,0,0],ds=[4,4,4])
model=syntheticModel.geoModel(hyper,{"vel":5000.},"vel")
depositC=syntheticModel.deposit()
depositI=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.)
depositN=syntheticModel.deposit(noiseDev=300.)
depositIC=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.,interbedDev=8.)
mt=depositC.apply(model,thick=40,prop=4700)
mt2=depositI.apply(mt,thick=70,prop=4100)
mt3=depositN.apply(mt2,thick=60,prop=3700)
mt4=depositIC.apply(mt3,thick=60,prop=3100,expand=False)
depositI=hv.Image(mt4.get().getNdArray()[5,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,1000,400*4,0)).opts(invert_yaxis=True,aspect=2,\
            toolbar=None,cmap="gist_rainbow",clim=(3100,5000))
hv.renderer("bokeh").save(depositI,"deposit",fmt="png")
depositI

In [ ]:
mtS=syntheticModel.squish().apply(mt3)
sq=hv.Image(mtS.get().getNdArray()[5,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,1000,400*4,0)).opts(invert_yaxis=True,aspect=2,\
            toolbar=None,cmap="gist_rainbow",clim=(3100,5000))
hv.renderer("bokeh").save(sq,"squish",fmt="png")
sq

In [ ]:
mtSalt=syntheticModel.salt().apply(mtS,seed=23,beg_freq=.3,\
            salt_value={"vel":4500})
sa=hv.Image(mtSalt.get().getNdArray()[5,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,1000,400*4,0)).opts(invert_yaxis=True,aspect=2,\
            toolbar=None,cmap="gist_rainbow",clim=(3100,5000))
hv.renderer("bokeh").save(sa,"salt",fmt="png")
sa

In [ ]:
mtC=syntheticModel.chimneyBig().apply(mtS)
mT=depositC.apply(mtC,prop=1500,thick=40)
ch=hv.Image(mT.get().getNdArray()[5,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,1000,400*4,0)).opts(invert_yaxis=True,aspect=2,\
            toolbar=None,cmap="gist_rainbow",clim=(3100,5000))
hv.renderer("bokeh").save(ch,"chimney",fmt="png")
ch

In [ ]:

hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,800,100],os=[0,0,0],ds=[4,4,4])
model=syntheticModel.geoModel(hyper,{"vel":5000.},"vel")
depositC=syntheticModel.deposit()


depositI=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.)


depositN=syntheticModel.deposit(noiseDev=300.)
depositIC=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.,interbedDev=8.)
#mt=depositC.apply(model,thick=40,prop=4700)
#mt2=depositI.apply(mt,thick=70,prop=4100)
#mt3=depositN.apply(mt2,thick=60,prop=3700)
mt4=depositIC.apply(model,thick=60,prop=3100,noiseDev=150.)
squishMe=syntheticModel.squish(begFreq=9.,maxShift=60)

mt0=squishMe.apply(mt4)
mt1=depositIC.apply(mt0,thick=70,prop=2900.,noiseDev=70.)
mt2=depositN.apply(mt1,thick=80,prop=2700)

#hv.renderer("bokeh").save(deposit,"deposit",fmt="png")
#deposit

In [ ]:
mtS=squishMe.apply(mt2,maxShift=400,begFreq=22.,seed=0)

img=hv.Image(mtS.get().getNdArray()[50,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,4*mtS.get().getNdArray().shape[2],800*4,0)).opts(invert_yaxis=True,aspect=2,\
            cmap="gist_rainbow",clim=(2500,5000))
img

In [ ]:

dep2=syntheticModel.deposit(seed=2).apply(mtS,expand=True)
x=dep2.get().getNdArray()[50,:,:]
y=dep2.getIntField("layer").getNdArray()[50,18,:]
img2=hv.Image(dep2.get().getNdArray()[50,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,4*dep2.get().getNdArray().shape[2],800*4,0)).opts(invert_yaxis=True,aspect=2,\
            cmap="gist_rainbow",clim=(2500,5000))
img2

In [ ]:
slt=syntheticModel.salt(seed=24,beg2=0,end2=.5,salt_value={"vel":4500})
gg=slt.apply(dep2)
img3=hv.Image(gg.get().getNdArray()[50,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,4*gg.get().getNdArray().shape[2],800*4,0)).opts(invert_yaxis=True,aspect=2,\
            cmap="gist_rainbow",clim=(2500,10000))
img3

In [ ]:
outM=syntheticModel.fault().apply(gg,angle=30,radius=1000,indicateI=True,begy=.7,
                                  theta_shift=4,theta_die=13)
img3=hv.Image(outM.getFloatField("vel").getNdArray()[50,:,::-1].T,\
                 kdims=[("x","Position(m)"),("y","Depth(m)")],\
                 bounds=(0,4*gg.get().getNdArray().shape[2],800*4,0)).opts(invert_yaxis=True,aspect=2,\
            cmap="gist_rainbow",clim=(2000,5000))
img3

#x=outM.get().getNdArray()